In [11]:
from api.filters import get_filter_query
import requests

In [10]:

filters = {
    'numerical_range': [
        {
            'attribute_name': 'price',
            'min': 90.00,
            'max': 135.00
        },
        {
            'attribute_name': 'accommodates',
            'min': 3,
            'max': None
        }
    ],
    'region': {
        'region_type': 'neighborhood',
        'id': 35
    }
}
query = get_filter_query(filters)

In [36]:
query

<Q: (AND: ('price__lte', 99.0), ('accommodates__gte', 4))>

In [63]:
# Place GET request with filters attached to JSON body
r = requests.get('http://localhost:8000/api/neighborhoods/',) #json={'filters': filters})

In [67]:
results = r.json()
results['features'][35]['properties']

{'data': {'avg_estimated_bookings_per_listing': 18.533333333333335,
  'avg_listing_price': 84.0,
  'crime_count': 234,
  'listing_count': 45,
  'population_latimes_2000_census': 36817},
 'name': 'Monrovia'}

In [1]:
from django.contrib.gis.db.models.functions import AsGeoJSON
n = Neighborhood.objects.annotate(geojson=AsGeoJSON('mpoly')).first()

In [5]:
with open('../test.json', 'w') as f:
    f.write(n.geojson)

In [8]:
type(n.geojson)

str

In [5]:
l = Listing.objects.first()
l.amenities

<django.db.models.fields.related_descriptors.create_forward_many_to_many_manager.<locals>.ManyRelatedManager at 0x115eb4128>

In [6]:
getattr(Listing, 'amenities')

In [23]:
filters = {
    'region': {
        'region_type': 'neighborhood',
        'id': 35
    }
}
query = get_filter_query(filters)
r = requests.post('http://localhost:8000/api/listings/filter/', json={'filters': filters})

In [24]:
r=requests.get('http://localhost:8000/api/neighborhoods/')

In [28]:
Neighborhood.objects.get(name="Vernon").listing_set.first()

<Listing: 6132543 Room Available ($125.00)>

In [29]:
Crime.objects.filter(neighborhood=Neighborhood.objects.get(name="Downtown")).count()

47025

In [48]:
Listing.objects.filter(Q(host_name__contains='&') | Q(host_name__icontains=" and ")).aggregate(Avg('reviews_per_month'))

{'reviews_per_month__avg': 2.12242562430246}

In [44]:
Listing.objects.exclude(host_name__contains='&').aggregate(Avg('reviews_per_month'))

{'reviews_per_month__avg': 1.426660547523}

In [53]:
Listing.objects.order_by('-accommodates').first().accommodates

16

In [55]:
Listing.objects.filter(price__lt=20.00).aggregate(Avg('reviews_per_month'))

{'reviews_per_month__avg': 2.56827859317555}

In [56]:
Listing.objects.aggregate(Avg('reviews_per_month'))

{'reviews_per_month__avg': 1.45916998379981}

In [60]:
Neighborhood.objects.annotate(aveeno=F('computed_stats')).first().aveeno

{'avg_listing_price': 451.0, 'crime_count': 842, 'listing_count': 3}

In [61]:
Crime.objects.values('crime_code_desc').distinct()

<QuerySet [{'crime_code_desc': 'GRAND THEFT / AUTO REPAIR'}, {'crime_code_desc': 'FELONIES, MISCELLANEOUS: Perjury'}, {'crime_code_desc': 'NARCOTICS: Found Narcotics'}, {'crime_code_desc': 'VEHICLE BURGLARY: Attempt Auto/Passenger Van Burglary'}, {'crime_code_desc': 'ASSAULT, AGGRAVATED: ADW - OTHER DANGEROUS WEAPON'}, {'crime_code_desc': 'EMBEZZLEMENT, GRAND'}, {'crime_code_desc': 'NARCOTICS: Miscellaneous Felonies'}, {'crime_code_desc': 'VEHICLE - ATTEMPT STOLEN'}, {'crime_code_desc': 'FEDERAL OFFENSES WITHOUT MONEY: Immigration Laws - Illegal Entry'}, {'crime_code_desc': 'ASSAULT, NON-AGG: Child Assault'}, {'crime_code_desc': 'DEFRAUDING: Welfare Fraud (Fel Or Misd)'}, {'crime_code_desc': 'FEDERAL OFFENSES WITH MONEY: Counterfeiting U.S. Money'}, {'crime_code_desc': 'DEPUTY INVOLVED SHOOTING/SUSPECT DEAD'}, {'crime_code_desc': 'NARCOTICS: Marijuana (Felony) Hashish/Hashish Oil (Poss/Sales)'}, {'crime_code_desc': 'IMMIGRATION: False Documents-Use to Conceal True Citizenship-Felony'},

In [62]:
Listing.objects.filter(number_of_reviews=0).count()

5996